# CQA Project Setup

This notebook outlines the process of setting up CQA dependencies for the Conversational Assistant demo.

It goes through the following steps:
- Creating a CQA project and importing example data.
- Deploying a knowledge base for production use.

**Note**: these steps can also be achieved using Azure AI [Language Studio](https://language.cognitive.azure.com/).

The example data is in the context of a fictional outdoor company: Contoso Outdoors. The data includes questions related to rental and refund policies, annual sales, etc.

To use CQA in this project, the following lines in your `.env` file should be updated by the end of this notebook:
```
CQA_PROJECT_NAME="<cqa-project-name>"
CQA_DEPLOYMENT_NAME="<cqa-deployment-name>"
```

### Prerequisites:
CQA requires an [Azure AI Language resource](https://learn.microsoft.com/en-us/azure/ai-services/language-service/). [Create a Language Resource](https://portal.azure.com/#create/Microsoft.CognitiveServicesTextAnalytics) first, if necessary. Ensure that your Language resource has Custom Question Answering enabled. This can be verified on the resource's page in Azure portal: `Resource Management/Features`.

From your Language resource, obtain its `endpoint` and `api-key`.

Update your `.env` file:
```
LANGUAGE_ENDPOINT="<language-endpoint>"
LANGUAGE_API_KEY="<language-api-key>"
```

In [ ]:
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT License.
import sys
sys.path.append("..")
import os
from dotenv import load_dotenv
from cqa_client import CqaClient

load_dotenv()

LANGUAGE_ENDPOINT = os.environ.get("LANGUAGE_ENDPOINT")
LANGUAGE_API_KEY = os.environ.get("LANGUAGE_API_KEY")

# Update with desired names:
CQA_PROJECT_NAME = "contoso-outdoors-cqa"
CQA_DEPLOYMENT_NAME = "production"

CQA_CLIENT = CqaClient(
    endpoint=LANGUAGE_ENDPOINT,
    api_key=LANGUAGE_API_KEY
)

print(f"Language endpoint: {LANGUAGE_ENDPOINT}")


Create Project:

In [ ]:
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT License.
CQA_CLIENT.create_project(
    project_name=CQA_PROJECT_NAME
)
print(f"Project created: {CQA_PROJECT_NAME}")


Import Project Data:

In [ ]:
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT License.
import json

import_file = "data/cqa/import.json"
with open(import_file, 'r') as fp:
    import_json = json.load(fp)

print(f"Project name: {CQA_PROJECT_NAME}")

import_result = CQA_CLIENT.import_project(
    import_json=import_json,
    project_name=CQA_PROJECT_NAME
)

print(f"Import project {'succeeded' if import_result else 'failed'}")


List Projects:

In [ ]:
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT License.
print("CQA projects:")
projects = CQA_CLIENT.list_projects()
print(projects)


Deploy Knowledgebase:

In [ ]:
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT License.
print(f"Creating deployment: {CQA_DEPLOYMENT_NAME}")
deploy_result = CQA_CLIENT.deploy(
    project_name=CQA_PROJECT_NAME,
    deployment_name=CQA_DEPLOYMENT_NAME
)
print(f"Deployment {'succeeded' if deploy_result else 'failed'}")


List Deployments:

In [ ]:
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT License.
print("CQA deployments:")
deployments = CQA_CLIENT.list_deployments(
    project_name=CQA_PROJECT_NAME
)
print(deployments)


Test Deployment:

In [ ]:
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT License.
from language_http_utils import run_language_api

url = "".join([
    LANGUAGE_ENDPOINT,
    "/language/:query-knowledgebases"
])
query_parameters = {
    "api-version": "2023-04-01",
    "projectName": CQA_PROJECT_NAME,
    "deploymenName": CQA_DEPLOYMENT_NAME
}

def create_input(
    question: str
) -> dict:
    """
    Create JSON input for CQA runtime.
    """
    return {
        "question": question,
        "top": 1
    }

def call_runtime(
    question: str
) -> dict:
    """
    Call CQA runtime.
    """
    input_json = create_input(
        question=question
    )

    return run_language_api(
        url=url,
        api_key=LANGUAGE_API_KEY,
        json_obj=input_json,
        query_parameters=query_parameters
    )

# Update question to test:
question = "What is your rental policy?"

cqa_response = call_runtime(
    question=question
)

print(f"CQA Runtime Response:\n{cqa_response}")


## Next steps:
Update your `.env` file:
```
CQA_PROJECT_NAME="<cqa-project-name>"
CQA_DEPLOYMENT_NAME="<cqa-deployment-name>"
```

You may experiment updating your CQA project in [Language Studio](https://language.cognitive.azure.com/) as well.